*Inizialmente in questo notebook lavoreremo con alcuni file definiti "...actorsTweetsPre" che contengono i tweet scaricati prima dell'inizio dell'evento e con alcuni file definiti "...actorsTweets" che invece contengono i tweet scaricati durante l'evento, a seguito dell'unione di questi file lavoreremo con il dataset "complete_actors"*

## 1. Importiamo le librerie necessarie per la fase di pre-processing

In [1]:
import pandas as pd 
import re
from numpy import nan
import difflib
from datetime import datetime, timedelta
! pip install emoji --upgrade

     |████████████████████████████████| 194kB 4.5MB/s eta 0:00:01
  Stored in directory: /home/monica/snap/jupyter/6/.cache/pip/wheels/5e/97/3a/e0e549eb14694b1692e49b14fe385518d78ae22fcec2d1aa6b
Successfully built emoji
  Found existing installation: emoji 1.4.2
    Uninstalling emoji-1.4.2:
      Successfully uninstalled emoji-1.4.2


Utilizziamo la funzione set option della libreria Pandas che ci consente di visualizzare il testo completo

In [2]:
pd.set_option('display.max_colwidth', None)

## 2. Importiamo i  file json contenenti i tweet scaricati

Tre file sono relativi ai tweet scaricati nelle ore precedenti all'evento 

In [3]:
M_preActors =  pd.read_json(r"tweet/M_actorsTweetsPre.txt") #2806 rows × 7 columns
B_preActors =  pd.read_json(r"tweet/B_actorsTweetsPre.txt") #2996 rows × 7 columns
C_preActors =  pd.read_json(r"tweet/C_actorsTweetsPre.txt") #2861 rows × 7 columns

Concateniamo i 3 dataset ottenendo il dataset completo "tot_preactors"

In [4]:
tot_preactors = pd.concat([M_preActors,C_preActors,B_preActors]) #8645 rows × 7 columns

Ulteriori tre file sono relativi ai tweet scaricati durante l'evento

In [5]:
M_ActorsTweets=  pd.read_json(r"tweet/M_ActorsTweets") #139645 rows × 7 columns
B_ActorsTweets=  pd.read_json(r"tweet/B_ActorsTweets") #140216 rows × 7 columns
C_ActorsTweets=  pd.read_json(r"tweet/C_ActorsTweets") #142216 rows × 7 columns

Concateniamo i 3 dataset ottenendo un dataset completo "tot_actors"

In [6]:
tot_actors = pd.concat([M_ActorsTweets,C_ActorsTweets,B_ActorsTweets]) #421969 rows × 7 columns

Creiamo ora un dataframe "complete_actors" contenente tutti i tweet acquisiti prima dell'evento e durante l'evento e ne correggiamo l'ora 

In [7]:
complete_actors = pd.concat([tot_actors,tot_preactors]) #430740 rows × 7 columns
complete_actors['date'] = pd.DatetimeIndex(complete_actors['date']) + timedelta(hours=-7)

Mettiamo in ordine i tweet in base all'ora di pubblicazione ed eliminiamo i tweet duplicati tramite la funzione drop duplicates

In [8]:
complete_actors.sort_values(by = ['date'], inplace=True, ignore_index= True)
complete_actors.drop_duplicates(subset = ["user_id", "date", "text"], ignore_index= True, inplace=True)

## 3. Analisi descrittiva 
1. Numero di utenti singoli e di tweet pubblicati durante la serata
2. Numero user bot (tweet pubblicati allo stesso secondo) 
3. Numero tweet e retweet per utente pre eliminazione dei bot 
4. Numero tweet e retweet per utente post eliminazione dei bot 
5. Numero tweet e retweet per attore/attrice
6. Numero user bot (tweet complessivi per utente > 50) 

### 3.1 Numero di utenti singoli e di tweet pubblicati durante la serata

Contiamo il numero di singoli utenti presenti nel dataset "complete_actors" (90650) ed i rispettivi tweet pubblicati e salviamo in un file csv il dataset "user_twitter"

In [10]:
user_twitter  = complete_actors.groupby(["screen_name"]).count()
user_twitter.to_csv(r"results/user_twitter.csv")

### 3.2 Numero user bot (tweet pubblicati allo stesso secondo) 

Dopodiché contiamo quanti di questi utenti hanno pubblicato tweet alla stessa ora e allo stesso secondo, raggruppando sia per nome che per data. Affinché si trovino gli utenti che hanno pubblicato più tweet allo stesso secondo imponiamo la condizione: "_id" > 1, tali utenti rappresentano per noi dei bot che verranno eliminati successivamente

In [11]:
user_twitter_bot = complete_actors.groupby(["screen_name", "date"]).count()
user_bot = user_twitter_bot[user_twitter_bot['_id']>1] 
unique_users_bot = user_bot.groupby(["screen_name"]).count()
user_twitter_bot.reset_index(inplace = True)
user_bot.reset_index(inplace = True)

Con la seguente funzione contiamo quanti tweet sono stati pubblicati dagli user bot durante la serata, il totale risulta pari a 264, di questi 81 sono stati pubblicati nello stesso secondo

In [12]:
i = 0 
for bot in set(user_bot["screen_name"]):
    for utente in complete_actors["screen_name"]:
        if bot == utente: 
            i = i+1

In [13]:
i

264

### 3.3 Numero tweet e retweet per utente pre eliminazione dei bot 

Creiamo il dataset "retweet" che contiene i soli retweet presenti nel dataset "complete_actors" e salviamo in "tot_rt" il numero complessivo di retweet in tutto il dataset "complete_actors" prima che avvenga l'eliminazione dei bot

In [14]:
complete_actors['text'] = complete_actors['text'].str.lower()

In [15]:
retweet = complete_actors[complete_actors['text'].str.contains('\srt @|^rt @', regex=True)] 
tot_rt = retweet.count()

Creiamo successivamente due dataset "users_RT" e "user_TW" dove per ogni utente è presente rispettivamente il numero di retweet e di tweet pubblicati. Infine "count_TW_RT" rappresenta l'unione dei due dataset precedenti, in tale dataset è presente: il numero di retweet (NumberRetweet) e di tweet per utente (NumberTweet), la percentuale di retweet sul totale dei tweet pubblicati dal singolo utente (RetweetRate) ed infine la percentuale di retweet sul totale dei tweet pubblicati da tutti gli utenti (RetweetRate_tot). 

In [16]:
users_RT = retweet.groupby('screen_name').count()
users_TW = complete_actors.groupby('screen_name').count()
count_TW_RT = users_TW.merge(users_RT, how='left', left_on=['screen_name'], right_on=['screen_name'])
count_TW_RT  = count_TW_RT[['user_id_x','user_id_y']]
count_TW_RT['RetweetRate'] = (count_TW_RT['user_id_y']/count_TW_RT['user_id_x'])*100
count_TW_RT['RetweetRate_tot'] = (count_TW_RT['user_id_y']/ 145525)*100
count_TW_RT["diff"] = count_TW_RT['user_id_x'] - count_TW_RT['user_id_y']
count_TW_RT.reset_index(inplace=True)
count_TW_RT.rename(columns={"user_id_x":"NumberTweet", "user_id_y":"NumberRetweet", "screen_name":"UserName"},inplace=True)

# Procediamo con l'eliminazione degli utenti bot 

Prepariamo le colonne "date" e "text" e carichiamo il dataset "track_actors" che contiene la lista dei nomi degli attori. Tale dataset ci consentirà di runnare la funzione "trova", nello specifico quest'ultima valuta se all'interno del tweet viene menzionato l'attore/attrice, una volta trovato il nome e/o cognome dell'attore/attrice li inserisce in una nuova colonna "reg", in caso contrario tale cella rimane vuota. Tale funzione consente successivamente di mantenere nel dataset solo quei tweet in cui effettivamente è stato nominato l'attore o l'attrice, scartando pertanto tutti i tweet che sono stati scaricati erroneamente. 

In [17]:
complete_actors['date'] = pd.to_datetime(complete_actors['date'], format = '%Y-%m-%d %H-%m-%s')
complete_actors.sort_values(by = ['date'], inplace=True, ignore_index= True) 
complete_actors['text'] = complete_actors['text'].str.replace("\n"," ", regex=True)
complete_actors['text'] = complete_actors['text'].str.replace("#","", regex=True)
complete_actors['text'] = complete_actors['text'].str.replace("http\S+","", regex=True)

Facciamo in modo che tutto il testo della colonna "text" in complete_actors sia minuscolo poiché nel dataset "track_actors" tutte le parole chiave sono in minuscolo. In questo modo il confronto diventa più agevole.

In [18]:
complete_actors

,_id,date,user_id,username,screen_name,text,hashtags
0,6085da912ab79c00399db265,2021-04-25 14:09:31,29838664,b'77,bozzie_me,burn hollywood and the oscars to the ground.,[Oscars]
1,6085da922ab79c00399db266,2021-04-25 14:09:32,769726314097946624,AwkwardOcto,OctoTimeKeep,rt @deer_head_xiris: yeah wow let’s pay respect to his memory by profiting off of his likeness in the most environmentally harmful way poss…,[]
2,6085da962ab79c00399db268,2021-04-25 14:09:37,46250847,e-bóia,kyammi,rt @navycherub: love to have my death ghoulishly monetized via the most embarrassing scam concept in recent history,[]
3,6085da962ab79c00399db269,2021-04-25 14:09:37,525817082,Chris Bernardo,HisAirnessFan45,i think daniel kaluuya should win best supporting actor.oscars,[Oscars]
4,6085da982ab79c00399db26a,2021-04-25 14:09:38,950123075885981696,SirMunches,MunchesSir,"rt @jenn0wow: idk what’s worse the fact that it’s an nft, the fact that chadwick boseman’s death is being commodified, or that this is in e…",[]
...,...,...,...,...,...,...,...
145520,{'$oid': '608631362ab79c003a6fefdf'},2021-04-25 20:19:12,2980096259,ً,meisseeking,deserved,[]
145521,{'$oid': '608631362ab79c003a6fefe0'},2021-04-25 20:19:12,204970044,Chris Smith,tankytankerson,pretty sure they just keep giving frances mcdormand oscars because they're scared of her academyawards,[AcademyAwards]
145522,{'$oid': '608631362ab79c003a6fefdc'},2021-04-25 20:19:12,253349349,👩🏽‍💻,Ohh_Daisyy,rt @crystalejandro: the way black people run away when something is funny&gt;&gt;&gt;&gt;,[]
145523,{'$oid': '608631362ab79c003a6fefda'},2021-04-25 20:19:12,2898053882,𝘣𝘶𝘯𝘯𝘺𝘣𝘢𝘯𝘨 ᙏ̤̫,shawinkgirl,rt @gma: glenn close is a national treasure. dabutt oscars,"[DaButt, Oscars]"


In [19]:
track_actors = pd.read_csv(r"dataset/trackActors.csv", sep = ",")

In [20]:
def trova (riga):
    ltot=[]
    for x,y,z in zip(track_actors['name'], track_actors['surname'], track_actors['keys']):
        string_re = re.compile(f'(?P<Name>{x})|(?P<Surname>{y})|(?P<Keys>{z})')
        occurrence = string_re.search(riga)
        if occurrence:
            if (occurrence.group("Name")) != '[]':
                ltot += [x + ' ' + y]
            elif (occurrence.group("Surname")) != '[]':
                ltot += [x + ' ' + y]
            elif (occurrence.group("Keys")) != '[]':
                ltot += [x + ' ' + y]
    return (ltot)

In [21]:
complete_actors['reg'] = complete_actors['text'].apply(trova)

Creiamo un nuovo dataset "tweet_per_actors" tramite la funzione explode che crea una riga per ogni citazione ad attore che viene fatta all'interno del tweet ed eliminiamo tutti i tweet associati a un utente bot

In [22]:
tweet_per_actors = complete_actors.explode('reg')

Tramite dropna eliminiamo tutti quei tweet che non contengono citazioni ad alcun attore/attrice, i tweet che vengono eliminati perché non citano attori/attrici sono pari a 34352

In [23]:
tweet_per_actors.dropna(inplace=True)

In [24]:
null_tweet_eliminated = 156086 -121734
null_tweet_eliminated

34352

Eliminiamo ora tutti i tweet pubblicati da utenti bot che hanno tweettato nella stessa data (ora e secondi)

In [25]:
for bot in set(user_bot['screen_name']):
     for utente in tweet_per_actors["screen_name"]:
            if bot == utente: 
                tweet_per_actors = tweet_per_actors.drop(tweet_per_actors["screen_name"].loc[tweet_per_actors["screen_name"]==utente].index)

In [26]:
bot_tweet_eliminated= 121734-121493
bot_tweet_eliminated

241

### Assegnamo ad ogni attore/attrice presente nella colonna "reg" il codice IMDB corrispondente

In [27]:
def trova1 (riga):
    ltot=[]
    for x,y in zip(track_actors['name'], track_actors['surname']):
        if riga["name"]== x and riga['surname']== y:
            ltot += [x + ' ' + y]
    return (ltot)

In [28]:
track_actors['NameSur'] = track_actors.apply(trova1, axis= 1)

In [29]:
track_actors["NameSur"] = track_actors["NameSur"].astype(str)

In [30]:
track_actors['NameSur'] = track_actors['NameSur'].str.replace("\['","", regex=True)

In [31]:
track_actors['NameSur'] = track_actors['NameSur'].str.replace("\']","", regex=True)

In [32]:
def codice (riga):
    trovato=[]
    for i, j in zip(track_actors['NameSur'], track_actors['actorId']):
        if riga["reg"] == i: 
            return (j)

In [33]:
tweet_per_actors['actorId'] = tweet_per_actors.apply(codice, axis = 1)

Salviamo il nuovo dataset nel file "tweet_per_actors"

In [34]:
tweet_per_actors.to_csv(r"results/tweet_per_actors.csv")

### 3.4 Numero tweet e retweet per utente post eliminazione dei bot 

Creiamo il dataset "retweet1" che contiene i soli retweet presenti nel dataset "tweet_per_actors" e salviamo in "tot_rt1" il numero complessivo di retweet presenti in tutto il dataset "tweet_per_actors" (110258).

In [35]:
retweet1 = tweet_per_actors[tweet_per_actors['text'].str.contains('\srt @|^rt @', regex=True)] 
tot_rt1 = retweet1.count()

In [36]:
tot_rt1

_id            110258
date           110258
user_id        110258
username       110258
screen_name    110258
text           110258
hashtags       110258
reg            110258
actorId        110258
dtype: int64

Creiamo successivamente due dataset "users_RT1" e "users_TW1" dove per ogni utente è presente rispettivamente il numero di retweet e di tweet pubblicati. Infine "count_TW_RT1" rappresenta l'unione dei due dataset precedenti, in tale dataset è presente: il numero di retweet (NumberRetweet) e di tweet per utente (NumberTweet), la percentuale di retweet sul totale dei tweet pubblicati dal singolo utente (RetweetRate) ed infine la percentuale di retweet sul totale dei tweet pubblicati da tutti i tweet (RetweetRate_tot). 

In [37]:
users_RT1 = retweet1 .groupby(['screen_name', 'date']).count()
users_TW1 = tweet_per_actors.groupby(['screen_name', 'date']).count()
users_RT2 = users_RT1.groupby('screen_name').count()
users_TW2 =users_TW1.groupby('screen_name').count()
count_TW_RT1 = users_TW2.merge(users_RT2, how='left', left_on=['screen_name'], right_on=['screen_name'])

In [38]:
count_TW_RT1  = count_TW_RT1[['user_id_x','user_id_y']]
count_TW_RT1["RetweetRate"] = (count_TW_RT1['user_id_y']/count_TW_RT1['user_id_x'])*100
count_TW_RT1["RetweetRate_tot"] = (count_TW_RT1['user_id_y']/132025*100)
count_TW_RT1["diff"] = count_TW_RT1['user_id_x'] - count_TW_RT1['user_id_y']
count_TW_RT1.reset_index(inplace=True)
count_TW_RT1.rename(columns={"user_id_x":"NumberTweet", "user_id_y":"NumberRetweet","screen_name":"UserName"})
count_TW_RT1.to_csv(r"results/count_users_post_actors.csv")

### 3.5 Numero tweet e retweet per attore/attrice

Svolgiamo gli stessi procedimenti per gli attori, ottenendo un nuovo dataset "count_TW_RT_A" che presenta per ogni attore/attrice il numero di tweet e di retweet in cui è stato menzionato

In [39]:
number_RT_A = retweet1.groupby('reg').count()
number_TW_A = tweet_per_actors.groupby('reg').count()
count_TW_RT_A = number_RT_A.merge(number_TW_A, how='inner', on='reg')

In [40]:
count_TW_RT_A=count_TW_RT_A[['_id_x', '_id_y' ]]
count_TW_RT_A['RetweetRate'] = (count_TW_RT_A['_id_x']/count_TW_RT_A['_id_y'])*100
count_TW_RT_A["diff"] = count_TW_RT_A['_id_y'] - count_TW_RT_A['_id_x']
count_TW_RT_A.reset_index(inplace=True)
count_TW_RT_A = count_TW_RT_A.rename(columns={"_id_x":"NumberRetweet", "_id_y":"NumberTweet","reg":"ActorName"})

In [41]:
count_TW_RT_A.to_csv(r"results/count_actors_post.csv")

### 3.6 Numero user bot (tweet complessivi per utente > 50) 

Valutiamo ora gli utenti che potrebbero essere considerati bot qualora il numero di tweet pubblicati durante la serata sia considerevole. In "user_high_tweets" sono presenti i soli utenti che presentano un numero di tweet superiore a 50, poiché tali utenti non hanno pubblicato nel medesimo secondo e siccome rappresentano solo 9/90650 = 0.01% non si è ritenuto opportuno eliminarli

In [42]:
tweet_per_user = complete_actors.groupby("screen_name").count()

In [43]:
user_high_tweets = tweet_per_user[tweet_per_user["_id"]>50]

## 4. Valutazione utenti con screen_name o username costituiti da soli numeri

In [44]:
complete_actors[complete_actors['screen_name'].str.contains('^\d+$', regex=True)]
complete_actors[complete_actors['username'].str.contains('^\d+$', regex=True)]

,_id,date,user_id,username,screen_name,text,hashtags,reg
668,6085def6a7b11b003eb68922,2021-04-25 14:28:17,1300444418927812608,1111,ghostcowgoesboo,"rt @jenn0wow: idk what’s worse the fact that it’s an nft, the fact that chadwick boseman’s death is being commodified, or that this is in e…",[],[chadwick boseman]
3652,{'$oid': '6086051ca7b11b003a59bc52'},2021-04-25 17:11:03,1868204714,1111,angelemarie_,rt @variety: best supporting actor nominees daniel kaluuya and leslie odom jr. share a laugh on the oscars red carpet. we love to see it h…,[Oscars],"[daniel kaluuya, leslie odom]"
3775,{'$oid': '608605402ab79c003a6b8cda'},2021-04-25 17:11:38,83178391,555,SadeBrown_718,rt @crystalejandro: the way black people run away when something is funny&gt;&gt;&gt;&gt;,[],[]
7586,{'$oid': '6086097ca7b11b003a59fc94'},2021-04-25 17:29:43,1351973463557365760,2,kmk_299,now tell me why homophobe lakeith stanfield is there when he was trashing the organizations so days ago ?? oscars,[Oscars],"[andra day, lakeith stanfield]"
8052,{'$oid': '608609b82ab79c003a6bcf31'},2021-04-25 17:30:42,1173681213753765888,888,kozykid1,yeah,[],[]
...,...,...,...,...,...,...,...,...
138941,{'$oid': '60862fe9a7b11b003a5dead2'},2021-04-25 20:13:39,3208775450,3,Pitchayood,rt @geeksofcolor: congratulations to frances mcdormand on winning the academy award for best actress in a leading role for her performance…,[],[frances mcdormand]
139054,{'$oid': '60862fed2ab79c0039bac81e'},2021-04-25 20:13:43,954920996024152064,2013,O613_twt,"rt @gma: yuh-jung youn won the academy award for best supporting actress for her performance in ""minari"" at the 2021 oscars on sunday. htt…",[Oscars],"[andra day, yuh-jung youn]"
141109,{'$oid': '6086303d2ab79c003a6fc87b'},2021-04-25 20:15:03,330970436,0308,zno000,rt @mubi: best actress: frances mcdormand (nomadland) oscars,[Oscars],[frances mcdormand]
142961,{'$oid': '608630992ab79c003a6fd82e'},2021-04-25 20:16:36,64622856,3,DenimandChard,bitch please! i’m severely upset!,[],[]


## 5. Valutazione utenti con emoji nello username

In [45]:
complete_actors[complete_actors['username'].str.contains("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", regex=True)]

,_id,date,user_id,username,screen_name,text,hashtags,reg
35,6085dac22ab79c00399db2ab,2021-04-25 14:10:20,1301906563519721472,dudɑ ψ 📖: Mulherzinhas| six of crows,reggiesummer,rt @thefilmupdates: paul raci is the first nominee to hit the red carpet at the oscars nominated for best supporting actor in sound of me…,[Oscars],[paul raci]
52,6085dad62ab79c00399db2c9,2021-04-25 14:10:40,38705075,Michael Buckley 🌎💙,hello_buckers,"rt @jenn0wow: idk what’s worse the fact that it’s an nft, the fact that chadwick boseman’s death is being commodified, or that this is in e…",[],[chadwick boseman]
53,6085dadb2ab79c00399db2cd,2021-04-25 14:10:45,378787787,i want to bailey-ve 🛸,been_herde,rt @thefilmupdates: paul raci is the first nominee to hit the red carpet at the oscars nominated for best supporting actor in sound of me…,[Oscars],[paul raci]
56,6085dadf2ab79c00399db2d3,2021-04-25 14:10:49,1264611888161316864,💛SHAYNE💛 ATTITUDE ERA,PDSNPH,i will throw a chair at someone's head if she loses again i swear to god,[],[]
84,6085db032ab79c00399db30c,2021-04-25 14:11:25,3764922995,Sof ♀🏴,softlyburns,"rt @jenn0wow: idk what’s worse the fact that it’s an nft, the fact that chadwick boseman’s death is being commodified, or that this is in e…",[],[chadwick boseman]
...,...,...,...,...,...,...,...,...
145484,{'$oid': '608631342ab79c003a6fefab'},2021-04-25 20:19:10,1252809584214855680,B ^.^🐰,brooke12173,rt @variety: oscars: yuh-jung youn wins best supporting actress for minari,"[Oscars, Minari]",[yuh-jung youn]
145487,{'$oid': '608631342ab79c003a6fefae'},2021-04-25 20:19:10,250672954,Big Hero 6 🦉🐯,kitKAT_dc,rt @meridak: the oscars with a whole dc go-go moment. glenn close doing da butt. name-checking eu and backyard band? that’s mic-drop materi…,[],[glenn close]
145502,{'$oid': '608631342ab79c003a6fefbb'},2021-04-25 20:19:11,272209723,Moi🇯🇲,Unusual_Moi,rt @crystalejandro: the way black people run away when something is funny&gt;&gt;&gt;&gt;,[],[]
145522,{'$oid': '608631362ab79c003a6fefdc'},2021-04-25 20:19:12,253349349,👩🏽‍💻,Ohh_Daisyy,rt @crystalejandro: the way black people run away when something is funny&gt;&gt;&gt;&gt;,[],[]


## 6. Attraverso il file "track_actors" costruiamo un dizionario in cui ad ogni attore viene associato il numero totale di tweet in cui viene citato

Costruzione del dizionario track

In [46]:
track = track_actors['name'] +' '+ track_actors['surname']

In [47]:
numeroTweetAttori = {}
for name in set(track):
    numeroTweetAttori[name] = 0

In [48]:
for i in set(track):
    somma = tweet_per_actors['reg'].str.contains(i, case=False).sum()
    numeroTweetAttori[i] = somma

## 6.1 Trasformiamo il dizionario in un dataframe che definiamo "nTweetActors" in cui ad ogni attore/attrice è associato il numero totale di tweet in cui viene menzionato/a

In [49]:
nTweetActors = pd.DataFrame.from_dict(numeroTweetAttori, orient='index')
nTweetActors.reset_index(inplace=True)
nTweetActors.columns = ['Actors', 'nTweet']
nTweetActors.sort_values("Actors")

,Actors,nTweet
19,amanda seyfried,789
17,andra day,2478
12,anthony hopkins,1828
11,carey mulligan,2329
18,chadwick boseman,4230
7,daniel kaluuya,33795
0,frances mcdormand,5255
16,gary oldman,185
6,glenn close,21705
10,lakeith stanfield,4181


In [50]:
nTweetActors.to_csv(r"results/nTweetActors.csv")